# Multimodal Data Analysis with LLMs and Python

- [LLMs in Action](https://itrummer.github.io/DataScienceWithGPT/)
- [Data Scienece With GPT](https://github.com/itrummer/DataScienceWithGPT.git)

In [10]:
%%capture 
!pip install openai
!pip install -q pandas

## Setup

In [54]:
import os
import openai
from getpass import getpass
import sqlite3
from typing import Optional

In [3]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI_API_KEY: ")

Enter your OPENAI_API_KEY:  ········


## Analysis of Text
### Sentiment Analysis and Text Classification

In [4]:
client = openai.OpenAI()

In [5]:
def create_prompt(text):
    """Create the input prompt for the language model

    Args:
        text: text to classify.
    Returns:
        prompt for the text classification.
    """
    instructions = "Is the underlying sentiment positive or negative?"
    formatting = "Postive or Negative"

    return f"Text: {text}\n{instructions}\nAnswer ({formatting}):"

In [6]:
def call_llm(prompt):
    """Call the llm with input prompt and return answer.

    Args:
        prompt: prompt to send to the language model

    Returns:
        answer: answer generated by llm
    """
    messages = [
        {"content": prompt, "role": "user"}
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-4o",
    )
    return response.choices[0].message.content

In [7]:
text = "The movie was great."
prompt = create_prompt(text)
print(prompt)

Text: The movie was great.
Is the underlying sentiment positive or negative?
Answer (Postive or Negative):


In [8]:
answer = call_llm(prompt)
answer

'Positive'

### Analyzing Text Data from CSVs

In [11]:
import pandas

In [12]:
def classify(text):
    """Classify the input text

    Args:
        text: the text to classify
    Retunrs:
        Class label ("Positive" or "Negative").
    """
    prompt = create_prompt(text)
    answer = call_llm(prompt)
    return answer
    

In [14]:
path = "./text-data/reviews.csv"
df = pandas.read_csv(path)
df["class"] = df["text"].apply(classify)

In [15]:
df

,text,sentiment,class
0,"My daughter liked it but I was aghast, that a ...",neg,Negative
1,I... No words. No words can describe this. I w...,neg,Negative
2,this film is basically a poor take on the old ...,neg,Negative
3,"This is a terrible movie, and I'm not even sur...",neg,Negative
4,First of all this movie is a piece of reality ...,pos,Positive
5,"As a young lass, beautiful Joan Woodbury (as R...",neg,Positive
6,"Low budget, but still creepy enough to hold yo...",neg,Positive
7,"Just to save you the $3, or whatever it costs ...",neg,Negative
8,If you want to see a film starring Stan laurel...,pos,Positive
9,I am willing to tolerate almost anything in a ...,neg,Negative



## Analyzing Images

In [26]:
def analyze_image(image_url, question):
    """Answer question about the input image

    Args:
        image_url: URL of the image to analyze
        question: obtain answer to this question
    Returns:
        Anser to the input question
    """
    message = {
        "role": "user",
        "content": [
            {"type": "text", "text": question},
            {"type": "image_url", "image_url": {"url": image_url}}
        ]
    }
    messages = [message]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


def analyze_images(image_url_1, image_url_2, question):
    """Answer question about the two input images

    Args:
        image_url_1: URL of the first image to analyze
        image_url_2: URL of the second image to analyze
        question: obtain answer to this question
    Returns:
        Anser to the input question
    """
    message = {
        "role": "user",
        "content": [
            {"type": "text", "text": question},
            {"type": "image_url", "image_url": {"url": image_url_1}},
            {"type": "image_url", "image_url": {"url": image_url_2}}
        ]
    }
    messages = [message]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [17]:
image_url = "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit1.jpg"
question = "What is this?"
answer = analyze_image(image_url, question)
print(answer)

This is an image of apples growing on a tree.


In [25]:
image_urls = [
    "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit1.jpg",
    "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit2.jpg",
    "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit3.jpg",
    "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit4.jpg",
    "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit5.jpg",
]
for image_url in image_urls:
    question = "What is this?"
    answer = analyze_image(image_url, question)
    print(answer)

This is an image of apples growing on a tree.
This is an apple.
This is a bunch of bananas.
This is an illustration of an orange with slices and leaves. The orange is a citrus fruit known for its bright color and juicy, sweet-tart flavor.
This is an image of an orange, with one whole orange and a slice of it.


In [27]:

image_url_1 = "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit1.jpg"
image_url_2 = "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit2.jpg"
question = "Compare the two images"
answer = analyze_images(image_url_1, image_url_2, question)
print(answer)

The first image shows a cluster of apples on a tree, surrounded by green leaves. The apples are red and appear to be growing naturally in an outdoor setting.

The second image features a single apple against a plain white background. This apple is also red but has a more polished appearance, likely due to lighting and editing for a clean presentation.

Overall, the first image emphasizes a natural, organic setting, while the second image focuses on showcasing the apple itself in a more isolated and commercial manner.


In [30]:
from IPython.display import Markdown, display

image_url_1 = "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit3.jpg"
image_url_2 = "https://raw.githubusercontent.com/itrummer/DataScienceWithGPT/refs/heads/main/data/fruit5.jpg"
question = "Compare the two images"
answer = analyze_images(image_url_1, image_url_2, question)
display(Markdown(answer))

The first image shows a bunch of bananas, which are elongated and have a smooth, yellow peel. The second image features an orange, which is round with a textured, orange peel, and a slice of the orange showing its juicy interior. 

Key differences:
- **Shape**: Bananas are elongated, while oranges are round.
- **Color**: Bananas are yellow, and oranges are orange.
- **Texture**: Bananas have a smooth peel, whereas oranges have a textured peel.
- **Presentation**: The orange is shown whole and sliced, revealing the inside, while the bananas are shown whole.

## Analyzing Audio Data

In [34]:
def transcribe_audio(audio_path):
    """Transcribes audio files and returns text

    Args:
        audio_path: path to file to transcribe
    Returns:
        transcription
    """
    with open(audio_path, "rb") as audio_file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
        return response.text

In [35]:
audio_path = "audio-data/data_QuoteFromTheAlchemist.mp3"
response = transcribe_audio(audio_path)
print(response)

Two years ago, right here on this spot, I had a recurrent dream too.


In [36]:
audio_path = "audio-data/shecenter_trailer.mp3"
response = transcribe_audio(audio_path)
print(response)

Welcome to SHE Center, the female podcast. I am Resti Kansime, and my mission is to create a safe space for women to share their stories and triumphs. SHE Center is more than just a podcast. It's my purpose. I have partnered up with Brilliant Minds to build a digital platform that serves women in Uganda and the rest of East Africa, empowering them to thrive in all aspects of their lives. Through this podcast, I aim to bring healing and clarity to my female listeners by showcasing the power of storytelling. The episodes will feature women from diverse backgrounds sharing their personal experiences and lessons learned. While my focus is on Uganda and East Africa, I believe in the global sisterhood. So whether you're listening from Uganda, East Africa or anywhere in the world, join us on this journey of self-discovery, growth and empowerment. Let's amplify the voices of women, break barriers, challenge stereotypes and create a network of support and empowerment. Together, we can bring abo

## Analyzing Tables

In [74]:
import re


def extract_sql_structure(db_path):
    """Extracts create-tabke statements from the database.

    Args:
        db_path: path to SQLite database.
    Returns:
        create-table statements from database
    """
    with sqlite3.connect(db_path) as connection:
        cursor = connection.cursor()
        cursor.execute(
            "SELECT sql from sqlite_master where type='table'"
        )
        table_rows = cursor.fetchall()
        table_cells = [row[0] for row in table_rows]
        return "\n".join(table_cells)


def create_sql_prompt(question: str, db_structure: Optional[str] = None):
    """Crerates a prompt for text-to-SQL translation

    Args:
        question: question to translate.
    Returns:
        SQL query translating question
    """
    parts = []
    parts.append("Database:")
    if db_structure is not None:
        parts.append(db_structure)
    else:
        parts.append("create table Customers(customerID int, customerName Text, City text);")
    parts.append("Translate this question to SQL:")
    parts.append(question)
    return "\n".join(parts)


def call_llm_sql(prompt):
    """Returns answer generated by the LLM.

    Args:
        prompt: prompt sent to the LLM.
    Returns:
        Answer generated by the LLM.
    """
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


def process_query(db_path, query):
    """Processes query on SQLite database

    Args:
        db_path: path to the SQlite database
        query: SQL query to process
    Returns:
        query result (as string)
    """
    with sqlite3.connect(db_path) as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        rows_str = [str(row) for row in rows]
        return "\n".join(rows_str)

In [57]:
question = "How many customers do we have?"
prompt = create_sql_prompt(question)
print(prompt)

Database:
create table Customers(customerID int, customerName Text, City text);
Translate this question to SQL:
How many customers do we have?


In [41]:
answer = call_llm_sql(prompt)
print(answer)

To find out how many customers you have, you can use the `COUNT` function in SQL to count the number of entries in the `Customers` table. Here is the SQL query for that:

```sql
SELECT COUNT(*) AS total_customers FROM Customers;
```


In [58]:
question = "Return the details of customers living in Kampala city."
prompt = create_sql_prompt(question)
print(prompt)

answer = call_llm_sql(prompt)
print(answer)

Database:
create table Customers(customerID int, customerName Text, City text);
Translate this question to SQL:
Return the details of customers living in Kampala city.
To retrieve the details of customers living in Kampala city, you can use the following SQL query:

```sql
SELECT * FROM Customers WHERE City = 'Kampala';
```


In [43]:
question = "Return the name of customer with ID 102."
prompt = create_sql_prompt(question)
print(prompt)

answer = call_llm_sql(prompt)
print(answer)

Database:
create table Customers(customerID int, customerName Text, City text);
Translate this question to SQL:
Return the name of customer with ID 102.
```sql
SELECT customerName FROM Customers WHERE customerID = 102;
```


In [44]:
question = "Return the details of customer with ID 102."
prompt = create_sql_prompt(question)
print(prompt)

answer = call_llm_sql(prompt)
print(answer)

Database:
create table Customers(customerID int, customerName Text, City text);
Translate this question to SQL:
Return the details of customer with ID 102.
To retrieve the details of a customer with a specific ID from the `Customers` table, you can use the following SQL query:

```sql
SELECT * FROM Customers WHERE customerID = 102;
```


In [45]:
display(Markdown(answer))

To retrieve the details of a customer with a specific ID from the `Customers` table, you can use the following SQL query:

```sql
SELECT * FROM Customers WHERE customerID = 102;
```

In [49]:
sql_queries = re.findall("```sql(.*)```", answer, re.DOTALL)
print(sql_queries[0])


SELECT * FROM Customers WHERE customerID = 102;



### Analyzing table data directly from SQL database

In [60]:
db_path = ("sql-data/games.db")
db_structure = extract_sql_structure(db_path)
print(f"DB structure:\n{db_structure}\n")
question = "How many video games are stored in the database?"
prompt = create_sql_prompt(question=question, db_structure=db_structure)
print(f"SQL prompt:\n{prompt}\n")
raw_answer = call_llm_sql(prompt)
sql_queries = re.findall("```sql(.*)```", raw_answer, re.DOTALL)
print(sql_queries[0])

DB structure:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)

SQL prompt:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
How many video games are stored in the database?


SELECT COUNT(*) AS total_games FROM games;



In [78]:
question = "Return rank, name, platform, genre, and year of games released in 2020. Limit the results to 10"
prompt = create_sql_prompt(question=question, db_structure=db_structure)
print(f"--- SQL Prompt ---:\n{prompt}\n")
raw_answer = call_llm_sql(prompt)
sql_queries = re.findall("```sql(.*)```", raw_answer, re.DOTALL)
query = sql_queries[0]
print(f"--- SQL QUERY ---:\n{query}\n")
result = process_query(db_path, query)
print(f"--- QUERY Result ---:\n{result}\n")

--- SQL Prompt ---:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
Return rank, name, platform, genre, and year of games released in 2020. Limit the results to 10

--- SQL QUERY ---:

SELECT rank, name, platform, genre, year
FROM games
WHERE year = 2020
LIMIT 10;


--- QUERY Result ---:
(5959, 'Imagine: Makeup Artist', 'DS', 'Simulation', 2020)



In [72]:
question = "How many video games are stored in the database?"
prompt = create_sql_prompt(question=question, db_structure=db_structure)
print(f"--- SQL Prompt ---:\n{prompt}\n")
raw_answer = call_llm_sql(prompt)
sql_queries = re.findall("```sql(.*)```", raw_answer, re.DOTALL)
query = sql_queries[0]
print(f"--- SQL QUERY ---:\n{query}\n")
result = process_query(db_path, query)
print(f"--- QUERY Result ---:\n{result}\n")

--- SQL Prompt ---:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
How many video games are stored in the database?

--- SQL QUERY ---:

SELECT COUNT(*) AS total_games FROM games;


--- QUERY Result ---:
16598



In [80]:
question = "Return rank, name, platform, genre, and year of games released ordered in ascending order. Limit the results to 10"
prompt = create_sql_prompt(question=question, db_structure=db_structure)
print(f"--- SQL Prompt ---:\n{prompt}\n")
raw_answer = call_llm_sql(prompt)
sql_queries = re.findall("```sql(.*)```", raw_answer, re.DOTALL)
query = sql_queries[0]
print(f"--- SQL QUERY ---:\n{query}\n")
result = process_query(db_path, query)
print(f"--- QUERY Result ---:\n{result}\n")

--- SQL Prompt ---:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
Return rank, name, platform, genre, and year of games released ordered in ascending order. Limit the results to 10

--- SQL QUERY ---:

SELECT rank, name, platform, genre, year
FROM games
ORDER BY year ASC
LIMIT 10;


--- QUERY Result ---:
(259, 'Asteroids', '2600', 'Shooter', 1980)
(545, 'Missile Command', '2600', 'Shooter', 1980)
(1768, 'Kaboom!', '2600', 'Misc', 1980)
(1971, 'Defender', '2600', 'Misc', 1980)
(2671, 'Boxing', '2600', 'Fighting', 1980)
(4027, 'Ice Hockey', '2600', 'Sports', 1980)
(5368, 'Freeway', '2600', 'Action', 1980)
(6319, 'Bridge', '2600', 'Misc', 1980)
(6898, 'Checkers', '2600', 'Misc', 1980)
(240, 'Pitfall!', '2600', 'Platform', 1981)



In [82]:
### Iterative (Loop) questions
while True:
    question = input("Enter a question:")
    if question.lower() in ["quit", "exit", "bye"]:
        print("Good Bye!")
        break
    prompt = create_sql_prompt(question=question, db_structure=db_structure)
    print(f"--- SQL Prompt ---:\n{prompt}\n")
    raw_answer = call_llm_sql(prompt)
    sql_queries = re.findall("```sql(.*)```", raw_answer, re.DOTALL)
    query = sql_queries[0]
    print(f"--- SQL QUERY ---:\n{query}\n")
    result = process_query(db_path, query)
    print(f"--- QUERY Result ---:\n{result}\n")
    print("=" * 100)

Enter a question: How many games are stored in the database?


--- SQL Prompt ---:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
How many games are stored in the database?

--- SQL QUERY ---:

SELECT COUNT(*) AS total_games FROM games;


--- QUERY Result ---:
(16598,)



Enter a question: Which game had the most sales in the EU?


--- SQL Prompt ---:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
Which game had the most sales in the EU?

--- SQL QUERY ---:

SELECT name, platform, year, genre, publisher, eusales
FROM games
ORDER BY eusales DESC
LIMIT 1;


--- QUERY Result ---:
('Wii Sports', 'Wii', 2006, 'Sports', 'Nintendo', 29.02)



Enter a question: Break down the number of games by the genre


--- SQL Prompt ---:
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question to SQL:
Break down the number of games by the genre

--- SQL QUERY ---:

SELECT genre, COUNT(*) AS number_of_games
FROM games
GROUP BY genre;


--- QUERY Result ---:
('Action', 3316)
('Adventure', 1286)
('Fighting', 848)
('Misc', 1739)
('Platform', 886)
('Puzzle', 582)
('Racing', 1249)
('Role-Playing', 1488)
('Shooter', 1310)
('Simulation', 867)
('Sports', 2346)
('Strategy', 681)



Enter a question: quit


Good Bye!
